# Inference example with a Cellpose model: www.cellpose.org
The model is based on U-Net, however rather than training it directly on bitmask targets they first convert them to "spatial flows" representations and train on that. This makes segmentation of dense and touching cells more reliable. For details and additional tricks they use see the paper "Cellpose: a generalist algorithm for cellular segmentation".

To train it I used the script provided in the cellpose repo ie: `python -m cellpose --train ...` after I converted the dataset to the input format it expects.

In inference I just submit the masks as they were returned from the model - no postprocessing

In [ ]:
!pip install --no-index ../input/cellpose-install/cellpose-0.7.2-py3-none-any.whl --find-links=../input/cellpose-install

#### I'm running it in a separate process rather than as a regular notebook because I've faced issues with numpy version not updating after the `pip install` above. If you know how to fix this please let me know.

In [ ]:
# !cp -r ../input/cellpose-code ./
# %cd ./cellpose-code

In [ ]:
!mkdir /content
!cp '/kaggle/input/sartorius-cellpose-model/pseudo_3k_26_50_cellpose_residual_on_style_on_concatenation_off_train_2021_12_18_14_33_07.383445' /content

In [ ]:
%%writefile run.py
import torch
import numpy as np
from cellpose import models, io, plot
from pathlib import Path
import pandas as pd

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

test_dir = Path('/kaggle/input/sartorius-cell-instance-segmentation/test')
test_files = [fname for fname in test_dir.iterdir()]
model =  models.CellposeModel(gpu=True,\
                            pretrained_model=['/kaggle/input/sartorius-cellpose-final/fold0_cellpose_2021_12_29_04_58_56.251435_epoch_19',
                                              '/kaggle/input/sartorius-cellpose-final/fold1_cellpose_2021_12_29_09_09_40.859558_epoch_19',
                                              '/kaggle/input/sartorius-cellpose-final/fold2_cellpose_2021_12_29_09_22_20.324645_epoch_19',
                                              '/kaggle/input/sartorius-cellpose-final/fold3_cellpose_2021_12_29_09_35_14.250530_epoch_19',
                                              '/kaggle/input/sartorius-cellpose-final/fold4_cellpose_2021_12_29_09_48_19.213322_epoch_19',
                                             ])
size_model = torch.load('/kaggle/input/sartorius-cellpose-model/pseudo_size_model.pth')
ids, masks = [],[]
for fn in test_files:
    test_image = io.imread(str(fn))
    predicted_diams, diams_style = size_model.eval(test_image, channels=[0,0])
    preds, flows, _ = model.eval(test_image, diameter=predicted_diams, channels=[0,0],\
                                 augment=True, resample=True,)
#                                  diam_threshold=12, min_size=14,\
#                                  flow_threshold=0.4, mask_threshold=0.1)
    for i in range (1, preds.max() + 1):
        ids.append(fn.stem)
        masks.append(rle_encode(preds == i))
        
pd.DataFrame({'id':ids, 'predicted':masks}).to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
!python run.py

In [ ]:
# %cd ..
# !rm -r cellpose-code

In [ ]:
import pandas as pd
pd.read_csv('submission.csv')